In [1]:
# seeds 
seed_import = 265
seed_QSGD = 155
seed_customFreq_2_5 = 5454
seed_Custom_linear_16 = 38763
seed_Custom_linear = 5425
seed_FP8 = 29855
seed_FL_implementation = 546984

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime
import random


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int
np.random.seed(seed_import)

In [3]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Existing Quantization schemes

In [25]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.size)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), arr.flatten()))

QSGD
Before Quantization:  [6.2010784  5.162569   0.76865226 7.466602   3.740111   1.1433895
 5.6046157  4.0482373  1.5089221  6.1559772  5.7501736  1.437814
 1.4942687  4.8450994  2.201438   1.0919528 ]
Encoded b'$\x00\x00\x00\xde\x9b\x88A"e2Y\x01\x00\x00\x00'
After Quantization:  [8.5380535 8.5380535 0.        8.5380535 0.        0.        8.5380535
 0.        0.        8.5380535 0.        0.        0.        0.
 0.        0.       ]
52.88461538461539% smaller 
70.83333333333333% smaller 


AttributeError: 'bytes' object has no attribute 'size'

In [5]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), arr.flatten()))

GZip
Before Quantization:  [0.90765774 6.3578563  4.288546   9.938401   6.9905796  1.4159056
 5.0692506  6.502736   5.91606    0.42416796 9.026106   7.16679
 8.033768   8.928214   5.7196164  2.944929  ]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\nc`\x00\x01F8\xf9\xec\xe3\tG\x00&Q\x9a\x1b\x14\x00\x00\x00'
After Quantization:  [ 0.        0.        0.        0.        0.        0.       25.118114
  0.        0.        0.        0.        0.       25.118114  0.
  0.        0.      ]
39.42307692307692% smaller 
62.5% smaller 


In [6]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [5.918381  4.9161267 6.548626  7.793331  3.8165953 3.9774828 4.6172633
 4.3716025 6.663812  8.756392  5.2282224 9.64102   3.2362638 7.5991445
 5.797482  4.3392563]
Encoded b'FEFHCDEDGIEICGFD'
After Quantization:  [ 6.   5.   6.   8.   3.5  4.   5.   4.   7.  10.   5.  10.   3.5  7.
  6.   4. ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [23]:

import cv2


class customCompression():
    def __init__(self, quantization_type = np.uint16):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[9.84652    0.30960885 9.666981   6.0016656  2.7343524  3.843075
 6.549565   0.31134966 9.98901    0.9618042  2.0463183  5.0283666
 1.2629457  9.092205   0.13976674 3.7084904 ]
Encoded [251   4 246 151  67  95 165   4 255  21  49 126  29 231   0  92]
After Quantization:  [9.83451193 0.29426467 9.64138951 5.97206364 2.72760709 3.80909261
 6.5128064  0.29426467 9.98900986 0.95088088 2.0323664  5.00645157
 1.25987674 9.06202227 0.13976674 3.69321916]
<class 'numpy.uint8'> 48.275862068965516% smaller 
<class 'numpy.uint8'> 28.571428571428573% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.0004188804891911883


In [8]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
120


# Custom Frequency Domain Quantization

In [9]:
from scipy.fftpack import dct, idct

class customFreq_1_point_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.72998416 0.15012904 0.609318   0.35949707 0.48061848 0.5968587
 0.8331491  0.684415   0.14251563 0.3263785  0.209497   0.17660174
 0.91490906 0.3702937  0.8996515  0.07344269]
Encoded [ 0.8438   0.4243   0.634    0.659    0.555    0.7295   0.8115   0.671
  0.05167  0.1837   0.2617   0.1703   0.8115   0.04092  0.3423  -0.4568 ]
After Quantization:  [0.73014855 0.15011206 0.60932326 0.35941502 0.48084113 0.5968043
 0.83323854 0.6845399  0.14260003 0.3263825  0.20942762 0.17665154
 0.914859   0.37029183 0.8996779  0.07343774]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  7.992142e-09


In [10]:
from scipy.fftpack import dct, idct

class customFreq_singleDCT():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_singleDCT()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.88454074 0.94910806 0.63969904 0.14578189 0.73760605 0.91342556
 0.51004684 0.42321226 0.44793537 0.11974435 0.26347664 0.9703822
 0.3024216  0.38510683 0.04192746 0.09441092]
Encoded [ 1.957    0.717    0.02681  0.2455  -0.1786   0.564    0.307   -0.1582
  0.04593 -0.3403  -0.4849   0.1311   0.1483  -0.1406   0.3286  -0.1137 ]
After Quantization:  [0.8844253  0.9490402  0.63969946 0.1458528  0.7375307  0.91327846
 0.5099968  0.42319524 0.44787288 0.11974167 0.26347134 0.97031415
 0.30233243 0.3850881  0.04188873 0.0943968 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  4.478531e-09


# Custom Freq 3 (CustomFreq + Custom2 but more sophisticated)

In [11]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)
        return enc
    
    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_3():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [np.uint32, np.uint16, np.uint8]):
        self.precision_levels = precision_levels
        self.linearQuant_0 = Linear_quant(self.precision_levels[0])
        self.linearQuant_1 = Linear_quant(self.precision_levels[1])
        self.linearQuant_2 = Linear_quant(self.precision_levels[2])
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        self.mean_importance = np.mean(importance)
        # if self.mean_importance > 0.1:
        enc = self.linearQuant_0.encode(weights_f)  # 8-bit
        precision = int(str(self.linearQuant_0.quantization_type).split('t')[-1].split('\'')[0])
        print(precision)

        # elif self.mean_importance > 0.01:
        #     enc = self.linearQuant_1.encode(weights_f)   # 4-bit
        #     precision = int(str(self.linearQuant_1.quantization_type).split('t')[-1].split('\'')[0])
        #     # enc = self.quantize(weights_f, 16)
        # else:
        #     enc = self.linearQuant_2.encode(weights_f)   # 2-bit
        #     precision = int(str(self.linearQuant_2.quantization_type).split('t')[-1].split('\'')[0])

        
        # Quantize frequency components
        # enc = self.quantize(enc, precision)
        return enc


    def decode(self, enc) :

        # if self.mean_importance > 0.1:
        weights_quantized = self.linearQuant_0.decode(enc)  # 8-bit
        # elif self.mean_importance > 0.01:
        #     weights_quantized = self.linearQuant_1.decode(enc)   # 4-bit
        # else:
        #     weights_quantized = self.linearQuant_2.decode(enc)   # 2-bit
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_3()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.83453816 0.17894903 0.9229297  0.2823284  0.53155774 0.39412
 0.99434584 0.67541474 0.0671     0.77614385 0.30689502 0.23505269
 0.6041794  0.8990225  0.7389961  0.18466611]
32
Encoded [4294967295 2400079396 4043284544 3365598943 2802972051 3174533139
 3798012799 3833107054  912434659 3254757457 1922536587 1603872493
 2870211594 2795410423 1354528071          0]
After Quantization:  [0.83453815 0.17894895 0.92292955 0.28232848 0.53155773 0.39412001
 0.9943457  0.6754147  0.06709994 0.7761438  0.3068949  0.23505277
 0.60417941 0.89902228 0.73899597 0.18466608]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  9.773225458817566e-15


# CustomFreq2.5

In [12]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.08457882 0.42124805 0.3722813  0.64964575 0.34334862 0.6431326
 0.10253259 0.2812138  0.0502326  0.5405608  0.54730844 0.63952416
 0.5464231  0.25400323 0.595145   0.9587929 ]
Encoded [0.03485 0.6196  0.4678  0.8525  0.497   0.7104  0.2769  0.306   0.2761
 0.6143  0.676   0.4934  0.3772  0.087   0.4792  0.4683 ]
After Quantization:  [0.08454148 0.4210815  0.37222186 0.649454   0.34334493 0.6431519
 0.10243323 0.28109092 0.05025066 0.5407508  0.5472521  0.63945705
 0.54629487 0.25399274 0.5950858  0.9586857 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.0652746e-08


# Custom Freq 2 (CustomFreq + Custom2) 
- best version

In [22]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_2()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.3272563  0.6674304  0.3790206  0.52399    0.5374151  0.43613315
 0.80953354 0.8252634  0.5917188  0.72885865 0.34596226 0.4524998
 0.46309808 0.12290666 0.5386271  0.280848  ]
Encoded [1800939297 3992408889 3552144978 3169134953 3834028445 3067162690
 4072886175 4294967295 2513373920 2654695443 1085496962 1172492048
 1335144009          0 1734729028  268688201]
After Quantization:  [0.32725629 0.66743033 0.37902053 0.52399002 0.53741506 0.43613316
 0.80953342 0.82526341 0.5917188  0.7288585  0.34596215 0.45249986
 0.46309805 0.12290663 0.53862704 0.280848  ]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  4.119950216836073e-15


# Selected Schemes

In [20]:
class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")
Main.include("src/compressors/fp8.jl")

number_of_selected_schemes = 4
column_names = ["accuracy","time","efficiency"]
metrics_cell = np.zeros((number_of_selected_schemes, len(column_names)), dtype=object)

# list = []
# list.append(dict)
# dict[qs]={{a,t,e}





#______________QSGD________________
class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        np.random.seed(seed_QSGD)
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle
        

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=seed_QSGD, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)

# cmp = GZip(1)

#______________FP8________________
class FP8(Compressor):
    def __init__(self):
        np.random.seed(seed_FP8)
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=seed_FP8, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
# cmp = FP8()




#______________Custom Linear________________
class Custom_linear():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(seed_Custom_linear)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        
    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec

# data_type = np.uint8
# cmp = Custom_linear(data_type)


class Custom_linear_16():
    def __init__(self, quantization_type = np.uint16):
        np.random.seed(seed_Custom_linear_16)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec






#______________Frequency Domain Quantization________________



class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(seed_customFreq_2_5)
        self.precision_levels = precision_levels
        self.linearQuant = Custom_linear()
        #quantization_level from 0 to 100  
              
    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
# cmp = customFreq_2_5()





In [16]:
from sklearn.preprocessing import MinMaxScaler

def get_time_efficiency_acc(cmp):

    df_acc_loss = pd.read_csv("E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_ACC_LOSS_sheet.csv")

    acc_loss = df_acc_loss[df_acc_loss['Epoch']==9]
    acc_loss = acc_loss.drop(['Epoch'], axis=1)
    df_time_eff = pd.read_csv("E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_TIME_EFF_sheet.csv")
    df_properties = pd.merge(acc_loss, df_time_eff, on='scheme_name')
    # print(df_properties)
    scaler = MinMaxScaler()
    df_properties_scaled = df_properties.copy()
    df_properties_scaled[['Acc', 'Time', 'Efficiency']] = scaler.fit_transform(df_properties_scaled[['Acc', 'Time', 'Efficiency']])
    weights = {
        'Acc': 0.5,
        'Time': 0.3,
        'Efficiency': 0.2
    }

    # Compute the combined measure
    df_properties_scaled['Combined_Score'] = (
        weights['Acc'] * df_properties_scaled['Acc'] +
        weights['Time'] * (1 - df_properties_scaled['Time']) +  
        weights['Efficiency'] * df_properties_scaled['Efficiency']
    )


    name = cmp.__class__.__name__

    time = df_properties[df_properties['scheme_name']==name]['Time'].values[0]
    acc = df_properties[df_properties['scheme_name']==name]['Acc'].values[0]
    eff = df_properties[df_properties['scheme_name']==name]['Efficiency'].values[0]

    combined = df_properties_scaled[df_properties_scaled['scheme_name']==name]['Combined_Score'].values[0]


    return name, acc, time, eff, combined
    
    


# Federated Implementation

In [38]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp_list, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		
		np.random.seed(seed_FL_implementation)
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp = None
		self.cmp_list = cmp_list
		self.client_ids = list(range(0,number_of_clients))

		self.bw_low = 20		
		self.bw_high = 2500

		self.traffic_low = 1
		self.traffic_high = 51
		# self.bw_traffic = [[random.randint(self.bw_low, self.bw_high), random.randint(1,50)] for _ in range(number_of_clients)]
	
		bandwidths = np.random.randint(self.bw_low, self.bw_high + 1, size=self.number_of_clients)
		traffic = np.random.randint(self.traffic_low, self.traffic_high, size=self.number_of_clients)
		self.bw_traffic = list(zip(bandwidths, traffic))


#a list of tuple of shape(list) number_of_clients

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        

        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100

		weight_array = np.array(weights)
		try:
			assert(weight_array.shape == (6,2))
		except:
			print('Scheme',self.cmp)
			print('weight_array.shape',weight_array.shape)

		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights




	def get_client_bw_and_traffic(self,client_id):
		bw,traffic = self.bw_traffic[client_id]
		return bw,traffic
	


	def quanization_scheme(self,bw,traffic):
		normalized_bw = bw/(self.bw_high-self.bw_low)
		normalized_traffic = traffic/(self.traffic_high-self.traffic_low) 

		client_condition = normalized_bw * 50 + normalized_traffic * 50

		data = np.arange(0,100)
		percentile_33 = np.percentile(data, 33)
		percentile_66 = np.percentile(data, 66)

		if client_condition<=percentile_33:
			selected_cmp = self.cmp_list[0]
		elif percentile_33 < client_condition <= percentile_66:
			selected_cmp = self.cmp_list[1]
		else:
			selected_cmp = self.cmp_list[2]			
		return selected_cmp
	
		

	def client_generator(self, train_x, train_y):
		
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()		
		return weights, running_loss, start_size,end_size
	
	
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for selection_index,(dataset_x, dataset_y )in enumerate(clients):
				client_id = selections[selection_index]
				bw,traffic = self.get_client_bw_and_traffic(client_id)
				self.cmp = self.quanization_scheme(bw,traffic)
				# print(self.cmp)
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			self.test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(self.test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight
	
	def get_acc(self):
		return self.test_acc

	


#%%

In [39]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
# epoch_time = time.time()
tic = datetime.now()
filename = "saved_models_x"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)

cmp_list = [customFreq_2(),QSGD(2,True),Custom_linear_16()]
# cmp_list = [customFreq_2(),customFreq_2(),Custom_linear_16()]
d = {}

for cmp in cmp_list:
    name, acc, time, eff, combined = get_time_efficiency_acc(cmp)
    d[cmp] = eff
    
print(d)
sorted_schemes = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
print(sorted_schemes.keys())
# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp_list=list(sorted_schemes.keys()), filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", datetime.now()-tic)

{<__main__.customFreq_2 object at 0x0000024FB139E800>: 0.2758620689655172, <__main__.QSGD object at 0x0000024FC38E37C0>: 0.5288461538461539, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>: 0.4137931034482758}
dict_keys([<__main__.QSGD object at 0x0000024FC38E37C0>, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>, <__main__.customFreq_2 object at 0x0000024FB139E800>])


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:22<00:00,  7.26it/s]


Test Accuracy: 0.10228
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 1.95148}: 100%|██████████| 164/164 [00:24<00:00,  6.78it/s]


Test Accuracy: 0.47029
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


...

Test Accuracy: 0.86199
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.53745}: 100%|██████████| 164/164 [00:23<00:00,  7.00it/s]


Test Accuracy: 0.86518
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.50526}: 100%|██████████| 164/164 [00:27<00:00,  5.86it/s]


Test Accuracy: 0.94933
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  0:04:27.716845


In [ ]:
{<__main__.customFreq_2 object at 0x0000024FB139E800>: 0.2758620689655172, <__main__.QSGD object at 0x0000024FC38E37C0>: 0.5288461538461539, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>: 0.4137931034482758}
dict_keys([<__main__.QSGD object at 0x0000024FC38E37C0>, <__main__.Custom_linear_16 object at 0x0000024FC38E2E90>, <__main__.customFreq_2 object at 0x0000024FB139E800>])
  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
C:\Users\Asus\AppData\Local\Temp\ipykernel_7408\3597313781.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:22<00:00,  7.26it/s]
Test Accuracy: 0.10228
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 1.95148}: 100%|██████████| 164/164 [00:24<00:00,  6.78it/s]
Test Accuracy: 0.47029
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.74089}: 100%|██████████| 164/164 [00:26<00:00,  6.19it/s]
Test Accuracy: 0.73742
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.68466}: 100%|██████████| 164/164 [00:24<00:00,  6.76it/s]
Test Accuracy: 0.75613
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.61751}: 100%|██████████| 164/164 [00:22<00:00,  7.16it/s]
Test Accuracy: 0.83759
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.60218}: 100%|██████████| 164/164 [00:24<00:00,  6.57it/s]
Test Accuracy: 0.853
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.59334}: 100%|██████████| 164/164 [00:24<00:00,  6.79it/s]
Test Accuracy: 0.8574
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.58873}: 100%|██████████| 164/164 [00:27<00:00,  6.05it/s]
Test Accuracy: 0.86199
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53745}: 100%|██████████| 164/164 [00:23<00:00,  7.00it/s]
Test Accuracy: 0.86518
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.50526}: 100%|██████████| 164/164 [00:27<00:00,  5.86it/s]
Test Accuracy: 0.94933
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  0:04:27.716845

In [37]:
print(agg_weights)

print(m_8.get_acc())

None
tensor(0.9491)


# Q learning

In [1]:
# # seeds 
# seed_import = 265
# seed_QSGD = 155
# seed_customFreq_2_5 = 5454
# seed_Custom_linear_16 = 38763
# seed_Custom_linear = 5425
# seed_FP8 = 29855
# seed_FL_implementation = 546984

yes